## Improved Application for Demographics and Sentiment Prediction

### NOTE: Don't Forget to Activate OpenVINO Runtime
1. Open Command Line
2. Execute openvino_env\Scripts\activate

### Import Libraries

In [1]:
import sys
import time
import imutils
from operator import itemgetter

root_path = "c:/Users/Lenard/Person and Vehicle Counter/" #set to directory of Person and Vehicle Counter repository
sys.path += [root_path]

import cv2
import dnn
import dlib

from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image

from openvino.runtime import Core

import numpy as np
import math

from pyimagesearch.centroidtracker import CentroidTracker
from pyimagesearch.trackableobject import TrackableObject

### Define Video Dimensions

In [2]:
RES_W = 640 # 1280  # 640  # 256  # 320  # 480
RES_H = 480 # 960   # 480  # 192  # 240  # 360

### Get Face Detection Model

In [3]:
THRESH = 0.8 #minimum confidence score

detector = cv2.FaceDetectorYN.create(
        "face_detection_yunet_2022mar.onnx",
        "",
        (RES_W, RES_H),
        THRESH
)

### Get Classification Models (from Open Model Zoo/Intel Model)

In [4]:
## Define Labels
emotion_labels = ['neutral', 'happy', 'sad', 'surprise', 'anger']
gender_labels = ["female", "male"]

In [5]:
ie = Core()

#Age and Gender Model
#NOTE: change to path of saved model
age_gender_model = ie.read_model(model="c:/Users/Lenard/intel/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml")
compiled_age_gender_model = ie.compile_model(model=age_gender_model, device_name="GPU")
gender_output_layer = compiled_age_gender_model.outputs[0]
age_output_layer = compiled_age_gender_model.outputs[1]
age_gender_input_layer = next(iter(compiled_age_gender_model.inputs))

#Emotion Model
#NOTE: change to path of saved model
emotion_model = ie.read_model(model="c:/Users/Lenard/intel/emotions-recognition-retail-0003/FP32/emotions-recognition-retail-0003.xml")
compiled_emotion_model = ie.compile_model(model=emotion_model, device_name="GPU") 
emotion_output_layer = next(iter(compiled_emotion_model.outputs))
emotion_input_layer = next(iter(compiled_emotion_model.inputs))

### Get Centroid Tracker

In [6]:
ct = CentroidTracker(maxDisappeared=40, maxDistance=50)
trackers = []
trackableObjects={}

### Track Using Video

In [7]:
vid = cv2.VideoCapture(0)

#give camera time to warm up
time.sleep(0.1)

frame_count = 0

trackers = []
track_ids = []

#begin video capturing
while (vid.isOpened()):
    #capture frame
    ret, frame = vid.read()
    
    emotion_color = (0, 0, 0)
    gender_color = (0, 0, 0)
    
    if ret == True:
    
        #resize frame
        frame = imutils.resize(frame, width=RES_W, height=RES_H)
        image = frame

        # Loop over frames from the video stream
        # Obtain the raw numpy array representing the image, then initialize the timestamp
        # and the occupied/unoccupied text
        image_copy = np.copy(image)
        
        # start time for FPS throughput estimator
        t1 = time.time()
        
        #set status to waiting
        status = 'waiting'
        rects = []
        
        
        #On first frame
        if frame_count % 10 == 0:
            # set status and initialize our new set of object trackers
            status = 'detecting'
            trackers = []
            
            #detect
            detector.setInputSize((RES_W, RES_H))
            bboxes = detector.detect(image_copy)
            
            # Loop through list and overlay green bboxes
            for idx, bbox in enumerate(bboxes[1]):
                #Convert coordinates from float to int
                coords = bbox[:-1].astype(np.int32)
                track_ids.append(idx)
                #display bbox
                cv2.rectangle(image_copy, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0,255,0), 3)
                (startX, startY, endX, endY) = (coords[0], coords[1], coords[0]+coords[2], coords[1]+coords[3])
                
                # Initialize the tracker on the first frame
                # Create the correlation tracker (needs initialization priort to usage)
                tracker = dlib.correlation_tracker()
                
                # Begin a track on face detected on first frame
                rect = dlib.rectangle(coords[0], coords[1], coords[0]+coords[2], coords[1]+coords[3])
                tracker.start_track(image_copy, rect)
                
                #Add the tracker to list of trackers for utilization in skipping frames
                trackers.append(tracker)
        
        #On subsequent frames
        else:
            #Allow tracking from the previous frame (tracks all detected faces)
            for track_id, tracker in zip(track_ids, trackers):
                tracker.update(image_copy)
                pos = tracker.get_position()
                
                #Unpack the position object
                startX = int(pos.left())
                startY = int(pos.top())
                endX = int(pos.right())
                endY = int(pos.bottom())
                
                #append to rects list
                rects.append((startX, startY, endX, endY))
                
                #Get ROI for classification
                roi_color = image_copy[startY:(startY+endY), startX:(startX+endX)]
                roi_emotion = cv2.resize(roi_color, (64,64), interpolation=cv2.INTER_AREA)
                roi_age_gender = cv2.resize(roi_color, (62,62), interpolation=cv2.INTER_AREA)
                
                input_emotion = np.expand_dims(roi_emotion.transpose(2,0,1), axis=0) #expand dimensions for proper prediction (N,C,H,W)
                
                input_age_gender = np.expand_dims(roi_age_gender.transpose(2,0,1), axis=0) #expand dimensions for proper prediction (N,C,H,W)

                
                #Age Prediction
                age_predict = compiled_age_gender_model(inputs=[input_age_gender])[age_output_layer]
                age_label = int(age_predict*100)
                cv2.putText(image_copy, "Age: " + str(age_label), (startX, startY-70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
                
                #Gender Prediction
                gender_predict = compiled_age_gender_model(inputs=[input_age_gender])[gender_output_layer]
                gender_index = np.argmax(gender_predict)
                gender_label = gender_labels[gender_index]
                if gender_label == "male":
                    gender_color = (255, 0, 0)
                elif gender_label == "female":
                    gender_color = (0, 0, 255)
                cv2.putText(image_copy, "Sex: " + str(gender_label), (startX, startY-40), cv2.FONT_HERSHEY_SIMPLEX, 1, gender_color, 2)
                
                
                #Emotion Prediction
                emotion_predict = compiled_emotion_model(inputs=[input_emotion])[emotion_output_layer]
                emotion_index = np.argmax(emotion_predict)
                emotion_label = emotion_labels[emotion_index]
                
                if emotion_label == 'happy' or emotion_label == 'surprise':
                    emotion_color = (0, 255, 0)
                elif emotion_label == 'neutral':
                    emotion_color = (255, 0, 0)
                else:
                    emotion_color = (0, 0, 255)
                cv2.putText(image_copy, emotion_label, (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 1, emotion_color, 2)
                
                
                #Draw the bounding box for the tracker
                cv2.putText(image_copy, "Person - " + str(track_id), (startX, startY-100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                cv2.rectangle(image_copy, (startX, startY), (endX, endY), (0,255,0), 3)
        
        #use centroid tracker
        objects = ct.update(rects)
        
        #loop over tracked objects
        for (objectID, centroid) in objects.items():
            #check for an existing object ID for a trackable object
            to = trackableObjects.get(objectID, None)
            
            #if there is no existing trackable object, create one
            if to is None:
                to = TrackableObject(objectID, centroid)
                
            else:
                to.centroids.append(centroid)
                
            #store trackable object in dictionary
            trackableObjects[objectID] = to
            
            #display text and circle relating to the tracked object
            cv2.putText(
                image_copy, "ID {}".format(objectID),
                (centroid[0]-10, centroid[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
            )
            cv2.circle(image_copy, (centroid[0],centroid[1]), 4, (0,255,0), -1)
            
        
        #Increment frame count
        frame_count += 1
        
        #display video stream
        cv2.imshow('Video', image_copy)

        if cv2.waitKey(1) & 0xFF == ord('q'): #press Q to quit
            break

#clear stream capture
vid.release()
cv2.destroyAllWindows()